In [1]:
!pip install scapy requests


In [3]:
from scapy.all import sniff, IP
import requests

# Your VirusTotal API key
API_KEY = '2399d639231bd6ff55ffd4282ee47b57f5e55d53e89342faa66e51504f480574'

# VirusTotal API base URL for IP address reports
VT_URL = "https://www.virustotal.com/vtapi/v2/ip-address/report"

# Function to check if an IP address is malicious using VirusTotal
def check_malicious_ip(ip):
    params = {'apikey': API_KEY, 'ip': ip}
    try:
        # Send request to VirusTotal API
        response = requests.get(VT_URL, params=params)
        result = response.json()
        
        if result.get('response_code') == 1:  # IP found in VirusTotal
            positives = len(result.get('detected_urls', []))  # Detected malicious URLs for this IP
            if positives > 0:
                print(f"[ALERT] VirusTotal flagged this IP as malicious: {ip} ({positives} detected malicious URLs)")
            else:
                print(f"[INFO] IP is clean: {ip}")
        else:
            print(f"[INFO] IP not found in VirusTotal database: {ip}")
    except Exception as e:
        print(f"Error checking IP with VirusTotal: {e}")

# Function to handle each packet captured
def packet_handler(packet):
    if packet.haslayer(IP):  # Check if the packet has an IP layer
        src_ip = packet[IP].src
        dst_ip = packet[IP].dst
        print(f"[INFO] Source IP: {src_ip}, Destination IP: {dst_ip}")
        
        # Check if the source and destination IPs are malicious
        check_malicious_ip(src_ip)
        check_malicious_ip(dst_ip)

# Start sniffing packets
def start_sniffing(interface="Wi-Fi"):
    print(f"[*] Starting IP monitoring on {interface}...")
    sniff(iface=interface, prn=packet_handler, store=False)

if __name__ == "__main__":
    start_sniffing(interface="Wi-Fi")


[*] Starting IP monitoring on Wi-Fi...
[INFO] Source IP: 104.208.16.91, Destination IP: 192.168.1.39
[INFO] IP is clean: 104.208.16.91
[ALERT] VirusTotal flagged this IP as malicious: 192.168.1.39 (12 detected malicious URLs)
[INFO] Source IP: 192.168.1.39, Destination IP: 104.208.16.91
[ALERT] VirusTotal flagged this IP as malicious: 192.168.1.39 (12 detected malicious URLs)
[INFO] IP is clean: 104.208.16.91
[INFO] Source IP: 192.168.1.39, Destination IP: 8.8.8.8
[ALERT] VirusTotal flagged this IP as malicious: 192.168.1.39 (12 detected malicious URLs)
[ALERT] VirusTotal flagged this IP as malicious: 8.8.8.8 (100 detected malicious URLs)
[INFO] Source IP: 20.195.84.16, Destination IP: 192.168.1.39
[INFO] IP is clean: 20.195.84.16
Error checking IP with VirusTotal: Expecting value: line 1 column 1 (char 0)
[INFO] Source IP: 8.8.8.8, Destination IP: 192.168.1.39
[ALERT] VirusTotal flagged this IP as malicious: 8.8.8.8 (100 detected malicious URLs)
Error checking IP with VirusTotal: Expe